 # Bibliothéque

In [3]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras import Sequential
from sklearn.model_selection import train_test_split
from keras_preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec

#Préparation les données

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
file = open('/content/drive/My Drive/Arabic Chatbot/Modéles/Enseignement/Base De Données/Ens_A.txt', 'rt', encoding='utf-8')
text = file.read()
A_data = text.split("\n")
A_data[0]=A_data[0].replace('\ufeff','')


file = open('/content/drive/My Drive/Arabic Chatbot/Modéles/Enseignement/Base De Données/Ens_Q.txt', 'rt', encoding='utf-8')
text = file.read()
Q_data = text.split("\n")
Q_data[0]=Q_data[0].replace('\ufeff','')

In [6]:
Q_data[1]

'كيف يتم تنظيم السنة التحضيرية'

Ajouter `<start>` et `<end>` à chaque question et réponse

In [7]:
corpus = Q_data + A_data
corp = []
for a in corpus:
    s = ['<start>']
    for i in text_to_word_sequence(a):
        s.append(i)
    s.append('<end>')
    corp.append(s)

In [9]:
corp[0]

['<start>', 'ما', 'هو', 'تعريف', 'السنتين', 'التحضيريتين', '<end>']

In [11]:
ft_model2 = Word2Vec(sentences=corp, size=256, window=5, sample=1e-2, negative=10, sg=0, workers=10, min_count=1,iter=30)
ft_model2.wv.index2word
ft_model2.wv.save_word2vec_format('ENS_W2V2.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
ft_model2['تعريف']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-2.79277086e-01,  2.33795717e-01, -1.51433811e-01,  1.22505808e-04,
        3.16836983e-02, -1.59542978e-01, -8.79104994e-03,  4.32927758e-01,
       -1.95508897e-02,  3.54866683e-02, -2.74385717e-02, -1.57335937e-01,
       -4.06851232e-01,  1.05754547e-01, -2.99339384e-01, -3.42528015e-01,
       -8.39599967e-02,  1.05047919e-01, -8.49906132e-02, -2.41965711e-01,
       -4.00108732e-02,  2.77012974e-01,  4.22606282e-02,  1.10057564e-02,
       -3.95803787e-02,  5.24716601e-02,  2.72790819e-01, -2.29241177e-02,
       -2.73396149e-02, -4.29787695e-01,  2.36432076e-01, -1.62198514e-01,
        3.98210526e-01, -4.12427306e-01,  2.63025731e-01, -8.45362768e-02,
        4.65147525e-01,  1.02140263e-01,  3.11570577e-02, -5.76955676e-01,
       -2.43457053e-02, -2.41307154e-01, -5.38733542e-01,  2.54932016e-01,
       -3.19972664e-01,  1.75521430e-02,  2.61840582e-01,  6.59832716e-01,
        1.55671574e-02,  1.65699840e-01, -2.42863074e-01,  1.91680804e-01,
       -1.87413573e-01,  

#Prétraitement

**Etape 1: TOKENIZE**


*   Son but c’est de symboliser les données, c'est à dire convertie le texte en valeurs numériques **; transformer chaque chaque mot en nombre ;** 
 

*   Cela permet au réseau neuronal d'effectuer des opérations sur les données d'entrée

**Etape 2: PAD**


*   Lorsque nous introduisons nos séquences d'ID de mot dans le modèle finale, les séquence doient avoir la même longueur.
 

*   Pour ce faire, un remplissage est ajouté à toute séquence plus courte que la longueur maximale (c'est-à-dire plus courte que la phrase la plus longue).

**Etape 3: Prétraitement**


*   La création d'un dictionnaire des `{mot:id}` pour les Questions et les réponses .

*   Transfomer toutes les Questions et les Réponses à des vecteur.
*   Calcule de la taille du vocabulaire Q et A.

*   la taille maximale des Q et A

In [15]:
def tokenize(x):
    tokenizer = Tokenizer(char_level=False,filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(x)
    
    return tokenizer.texts_to_sequences(x), tokenizer

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
        print('Length Max: {}'.format(length))
        print()
    return pad_sequences(x, maxlen=length, padding='post')

def preprocess(Q,A):
 
    preprocess_Q, Q_tk = tokenize(Q)
    preprocess_A, A_tk = tokenize(A)

    preprocess_Q = pad(preprocess_Q)
    preprocess_A = pad(preprocess_A)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_A = preprocess_A.reshape(*preprocess_A.shape, 1)

    return preprocess_Q, preprocess_A, Q_tk, A_tk




preproc_Q, preproc_A, Q_tokenizer, A_tokenizer = preprocess(Q_data, A_data)

X_train, X_test, Y_train, Y_test = train_test_split(preproc_Q,preproc_A,test_size=0.05)

print(Q_tokenizer.word_index) #Dict Q
print(A_tokenizer.word_index) #Dict A


max_Q_sequence_length = preproc_Q.shape[1]
max_A_sequence_length = preproc_A.shape[1]

Q_vocab_size = len(Q_tokenizer.word_index)+1
A_vocab_size = len(A_tokenizer.word_index)+1

print('\n')

print('Data Preprocessed')
print("Max Q sentence length:", max_Q_sequence_length)
print("Max A sentence length:", max_A_sequence_length)
print("Q vocabulary size:", Q_vocab_size)
print("A vocabulary size:", A_vocab_size)

Length Max: 22

Length Max: 23

{'في': 1, 'العلوم': 2, 'ما': 3, 'و': 4, 'سلك': 5, 'لسلك': 6, 'التقنيات': 7, 'عدد': 8, 'هل': 9, 'من': 10, 'الهندسة': 11, 'بمدينة': 12, 'بكلية': 13, 'توجد': 14, 'المدرسة': 15, 'للتكنولوجيا': 16, 'هي': 17, 'المتوفرة': 18, 'المتاحة': 19, 'الماستر': 20, 'كلية': 21, 'جدع': 22, 'المسالك': 23, 'والتقنيات': 24, 'المقاعد': 25, 'دبلوم': 26, 'متى': 27, 'بنسبة': 28, 'الوطنية': 29, 'الجامعي': 30, 'الاجازة': 31, 'الدبلوم': 32, 'كمياء': 33, 'العليا': 34, 'بعد': 35, 'المشترك': 36, 'اعلاميات': 37, 'فيزياء': 38, 'رياضيات': 39, 'على': 40, 'التجارة': 41, 'الإجازة': 42, 'إجتياز': 43, 'للتسير': 44, 'الجذع': 45, 'السنتين': 46, 'الطالب': 47, 'الوحدة': 48, 'بسلك': 49, 'كيف': 50, 'مسلك': 51, 'التحضريتين': 52, 'جيولوجيا': 53, 'بيولوجيا': 54, 'المقاعدة': 55, 'لكلية': 56, 'والتقنات': 57, 'يتم': 58, 'السنة': 59, 'ماهي': 60, 'فصل': 61, 'الدكتوره': 62, 'ماذا': 63, 'هو': 64, 'كم': 65, 'ملال': 66, 'مدة': 67, 'لسنة': 68, 'طنجة': 69, 'التسجيل': 70, 'النقطة': 71, 'وحدة': 72, 'الاجازه': 73, '

In [18]:
Q_tokenizer.index_word[120]

'مشترك'

**Etape 4: Embbedings Questions**


*   l’utilisation d'une matrice qui contient les représentations des mots en vecteurs réalisés avec plusieurs méthodes **(W2V )** ,donc ça sera nécessaire de remplacer chaque mot par un vecteur comme input du model.


*   Et Dans notre cas la taille de chaque vecteur est 256

In [19]:

def embedding_matrix_creater(embedding_dimention):
  embedding_matrix = np.zeros((len(Q_tokenizer.word_index)+1, embedding_dimention))
  for word, i in Q_tokenizer.word_index.items():
    embedding_vector = None
    embedding_matrix[i] = ft_model2[word]
  return embedding_matrix


mat=embedding_matrix_creater(256)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [22]:
mat[1]

array([-0.46207306, -0.60303611,  1.54787183, -1.314762  ,  0.29274866,
       -1.31740439, -1.17480242,  0.84123272,  0.14033769,  1.98962426,
       -0.37804899,  1.03068137, -2.7585547 , -0.25747448, -1.16832483,
       -2.81849575,  0.7764253 , -1.31582308,  0.78062886,  1.92447531,
       -2.068156  , -1.2567718 ,  0.65682042,  1.0897038 ,  1.21606004,
        0.145658  ,  2.29095316, -1.50549471, -0.00681598,  0.47562578,
        2.48733354,  0.53210717,  0.14104395,  1.67774916, -2.65023828,
       -0.46176174,  1.38629925,  0.36206305,  1.85458374, -0.96838397,
        0.83814293,  0.03584776, -2.95593667, -0.06409174,  1.68396473,
        1.27854729,  1.49012721,  1.20457768,  0.2777983 ,  0.95744884,
        1.42743254,  1.82413077,  0.85401201,  1.2698648 ,  1.28267777,
       -0.2994214 , -0.96507442, -1.07278526, -0.84306449,  0.49596512,
       -1.07148528,  0.60592335,  0.32444298,  1.33332276,  0.63334876,
       -0.53730762,  0.66128534, -0.29320797, -1.01932943,  0.21

**Etape 4: Embbedings réponses**


*   l’utilisation d'une matrice qui contient les représentations des mots en vecteurs réalisés avec plusieurs méthodes **(W2V )** ,donc ça sera nécessaire de remplacer chaque mot par un vecteur comme input du model.


*   Et Dans notre cas la taille de chaque vecteur est 256

In [ ]:
def embedding_matrix_creater(embedding_dimention):
  embedding_matrix = np.zeros((len(A_tokenizer.word_index)+1, embedding_dimention))
  for word, i in A_tokenizer.word_index.items():
    embedding_vector = None
    embedding_matrix[i] = ft_model2[word]
  return embedding_matrix


mato=embedding_matrix_creater(256)

#Modèle

Encodeur - decodeur

In [ ]:
encoder_inputs = Input(shape=(max_Q_sequence_length, ))
input_embeeding = Embedding(input_dim=Q_vocab_size,
                            output_dim=256,
                            input_length=max_Q_sequence_length,
                            trainable=True,
                            weights=[mat],
                            name='enc_embedding1')(encoder_inputs)
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(input_embeeding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
dex=Embedding(input_dim=A_vocab_size,
                            output_dim=256,
                            trainable=True,
                            weights=[mato],
                            name='enc_embedding3')
output_embeeding = dex(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(output_embeeding,
                                     initial_state=encoder_states)
decoder_dense = Dense(A_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 24)           0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
enc_embedding1 (Embedding)      (None, 24, 256)      167424      input_9[0][0]                    
__________________________________________________________________________________________________
enc_embedding3 (Embedding)      (None, None, 256)    231168      input_10[0][0]                   
____________________________________________________________________________________________

Entrainement

In [ ]:
import keras.optimizers as optimizers
def generate_batch(X, y, batch_size = 32):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X)//batch_size*batch_size,batch_size):
            encoder_input_data = np.zeros((batch_size, max_Q_sequence_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_A_sequence_length-1),dtype='float32')
            decoder_output = np.zeros((batch_size, max_A_sequence_length-1, 903),dtype='float32')
            for i in range(batch_size):
                for c in range(max_Q_sequence_length):
                    encoder_input_data[i][c] = X[j+i][c]
                decoder_input_data[i] = np.reshape(y[j+i][0:max_A_sequence_length-1],(max_A_sequence_length-1))
                for c in range(max_A_sequence_length-1):
                    decoder_output[i][c][int(y[j+i][c+1][0])] = 1
            yield([encoder_input_data, decoder_input_data],decoder_output)

rms = optimizers.RMSprop(lr=0.005)
model.compile(optimizer=rms, loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(generate_batch(X_train,Y_train), epochs=60,steps_per_epoch=24)

c:\users\ucefb\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1/60
24/24 [==============================] - 8s 336ms/step - loss: 0.3909 - acc: 0.9063
Epoch 2/60
24/24 [==============================] - 7s 293ms/step - loss: 0.2650 - acc: 0.9350
Epoch 3/60
24/24 [==============================] - 7s 288ms/step - loss: 0.2129 - acc: 0.9470
Epoch 4/60
24/24 [==============================] - 8s 328ms/step - loss: 0.1811 - acc: 0.9536
Epoch 5/60
24/24 [==============================] - 7s 288ms/step - loss: 0.1596 - acc: 0.9570
Epoch 6/60
24/24 [==============================] - 7s 294ms/step - loss: 0.1415 - acc: 0.9612
Epoch 7/60
24/24 [==============================] - 8s 326ms/step - loss: 0.1256 - acc: 0.9639
Epoch 8/60
24/24 [==============================] - 7s 277ms/step - loss: 0.1117 - acc: 0.9672
Epoch 9/60
24/24 [==============================] - 7s 278ms/step - loss: 0.1025 - acc: 0.9692
Epoch 10/60
24/24 [==============================] - 6s 265ms/step - loss: 0.0917 - acc: 0.9722
Epoch 11/60
24/24 [==============================

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

output_emb = dex(decoder_inputs)


decoder_outputs2, state_h2, state_c2 = decoder_lstm(output_emb, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [ ]:
decoder_model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
enc_embedding3 (Embedding)      (None, None, 256)    231168      input_10[0][0]                   
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 256)          0                                            
____________________________________________________________________________________________

# Prédicter les réponses

toPredictible : tranforme les questions en vecteurs


get answer : prédicter la réponse 

In [ ]:
def toPredectible(qes, tok, shape):
    tt = []
    for q in qes.split(' '):
        if q in tok.word_index:
            tt.append(tok.word_index[q])
        else:
            tt.append(0)
    return pad([tt], shape)

def get_answer(query):
    query = '<start> '+query+' <end>'
    query = toPredectible(query,Q_tokenizer,24)
    
    #la première étape consiste à coder la question à l'aide du Encodeur 
    states_value = encoder_model.predict(query)
    
    #utilisez les états retournés par l'encodeur pour prédire le premier mot après le jeton <start>: preproc_A[-1][0] = 1 l'indice du jeton <start>
    output,H,C = decoder_model.predict( [preproc_A[-1][0]] + states_value)
    i = np.argmax(output[0][0])
    answer = A_tokenizer.index_word[i]
    
    #dans cette boucle, nous continuons à prédire le mot suivant:
    while True:
        output,H,C = decoder_model.predict( [np.array([i])] + [H,C])
        i = np.argmax(output[0][0])
        # vérifier si le jeton <end>(indice 2) est atteint : si oui = sortir de la boucle
        if i==2:
            break
        #concatenation
        answer =answer+' '+ A_tokenizer.index_word[i]
    return answer
